# Flower를 활용한 대규모 연합학습

Flower를 활용한 KAICD 연합학습 강좌 2부에 오신 것을 환영합니다!

이번 튜토리얼에서는 지난 시간에 구축한 연합학습 시스템을 커스터마이징 할 것입니다(이번에도 FLower와 PyTorch를 사용합니다!).

Part 1에서는 PyTorch를 활용하여 모델 훈련 파이프라인과 데이터 불러오기를 구성합니다.

Part 2에서는 Flower를 활용하여 Part 1에서 구현한 PyTorch 기반 파이프라인을 가지고 연합학습을 진행합니다.

## Part 0: 사전준비

시작하기에 앞서, Google Colab GPU 가속 설정을 확인해야 합니다.

`런타임 > 런타임 유형 변경 > 하드웨어 가속: GPU > 저장`

### 의존성 설치

다음으로 필요한 패키지를 설치하고 가져옵니다.

In [ ]:
!pip install torch==1.9.0 torchvision==0.10.0 git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

  Cloning https://github.com/adap/flower.git (to revision release/0.17) to /tmp/pip-install-o5dfdrvu/flwr_bc542f8c16e74044acdbc2f792938cf6
  Running command git clone -q https://github.com/adap/flower.git /tmp/pip-install-o5dfdrvu/flwr_bc542f8c16e74044acdbc2f792938cf6
  Running command git checkout -b release/0.17 --track origin/release/0.17
  Switched to a new branch 'release/0.17'
  Branch 'release/0.17' set up to track remote branch 'release/0.17' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Training on cuda:0


Google Colab에서 실행하고 런타임에 GPU 가속기가 있다면, 출력 결과에서 `Training on cuda:0` 문장을 확인할 수 있습니다.

### 데이터 불러오기

이제 CIFAR-10 훈련 및 테스트 데이터 세트를 불러오고, 10개의 작은 데이터 세트로 분할하여(각각 훈련 및 검증 데이터 세트로 분할) `DataLoader`로 포장합니다:

In [ ]:
NUM_CLIENTS = 10

def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()

Files already downloaded and verified
Files already downloaded and verified


### 모델 훈련/평가

기본적인 모델을 정의하고, 훈련 및 테스트 함수를 작성합니다:

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(testloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## 서버에서 클라이언트로 값 전송

TODO

In [ ]:
# TODO

## 클라이언트에서 서버로 값 전송

TODO

In [ ]:
# TODO

## 연합학습 전략 커스터마이징

TODO

## 요약

이번 튜토리얼에서는 클라이언트 측 실행을 완전히 커스터마이징하기 위해 서버와 클라이언트 간에 임의의 값들을 송수신할 수 있는 방법을 살펴봤습니다.

우리는 Flower의 가상 클라이언트 엔진을 사용하여 대규모 연합학습 시뮬레이션을 구축하고,

동일한 워크로드에 1000개의 클라이언트가 포함된 실험을 진행했습니다.

그리고 이 모든건 여기, 주피터 노트북 안에서 이루어졌죠!